In [1]:
import torch
from torch import nn
import numpy as np
import pandas as pd
from d2l import torch as d2l
#23/05/2023 - done with the most basic form of MLP implementation
#24/05/2023 - fixed wrong dimensions of inputs in Evaluation & Loss Functions and other bugs, added Dropout & Unflatten layer

In [2]:
#Config Parameters
quantiles = [0.05, 0.10, 0.20, 0.30, 0.40, 0.60, 0.70, 0.80, 0.90, 0.95]
qlen = len(quantiles)
i_train = 2400
i_val = i_train + 800
i_test = i_val + 1600 
#2400 for training, 800 for validation, 1600 for testing
batch_size, lr, n_epochs, num_iter = 256, 0.001, 100, 30
lag_period, num_features, forecast_horizon = 20, 14, 2
num_inputs, num_outputs, num_hidden = lag_period*num_features, (1+qlen)*num_features, 100
dropout= 0.7

In [3]:
#Load & Split Data
data = pd.read_csv("/Users/lixiang/Desktop/DeepJMQR Project/data.csv")
alldata = np.array(data)[:,1:].astype("float32")
torch.set_default_dtype(torch.float32)
X = []
for i in range(lag_period, len(data)):
    X.append(alldata[i-lag_period:i])
X = torch.tensor(np.array(X), requires_grad = True)
Y = alldata[lag_period+forecast_horizon:]

X_train, Y_train = X[:i_train], torch.tensor(Y[:i_train])
X_val, Y_val = X[i_train:i_val], Y[i_train:i_val]
X_test, Y_test = X[i_val:i_test], Y[i_val:i_test]
train_iter = torch.utils.data.DataLoader(list(zip(X_train,Y_train)), batch_size=batch_size, shuffle = True)
val_iter = torch.utils.data.DataLoader(list(zip(X_val,Y_val)), batch_size=batch_size, shuffle = False)

In [4]:
#Evaluation & Loss Functions
def lossfn(τ, y, ŷ):
    #for ConvLSTM & MLP
    #τ: quantile vector of length J
    #ŷ: prediction, 3D tensor of dim B x M x (1+J)
    #y: observation, 2D tensor of dim B x M
    #B = batch size or test/val data size
    loss = torch.sum(torch.square(y-ŷ[:,:,0]))
    for i in range(len(τ)):
        q = τ[i]
        r = y - ŷ[:,:,i+1]
        loss += torch.sum(q*r - r*(r<0))
    loss /= y.shape[0]
    return loss

#for evaluation: remember to turn tensors into np.array()
def tilted_loss(τ, y, ŷ):
    #y & ŷ: np.array() of same shape as defined in lossfn
    loss = 0.0
    for i in range(len(τ)):
        q = τ[i]
        r = y - ŷ[:,:,i+1]
        loss += np.sum(q*r - r*(r<0))
    loss /= y.shape[0]
    return loss
def crossing_loss(ŷ):
    #ŷ: np.array() of same shape as defined in lossfn
    loss = 0.0 #crossing loss as defined in the paper
    num_cross = 0.0
    for i in range(len(ŷ[0,0,:])-2):
        q = ŷ[:,:,i+1] - ŷ[:,:,i+2]
        loss += np.sum(np.maximum(q,0))
        num_cross += np.sum(q>0)
    loss /= ŷ.shape[0]
#     num_cross /= ŷ.shape[0]*ŷ.shape[1]*ŷ.shape[2]
    return loss, num_cross
def eval_quantiles(lower, upper, trues):
    #all inputs are np.array of dim B x M
    icp = np.mean((trues > lower) & (trues < upper))
    mil = np.mean(np.maximum(0,upper-lower))
    return icp,mil
def eval_error(y, ŷ):
    #y, ŷ: np.array() of same shape as defined in lossfn
    r = np.abs(y-ŷ[:,:,0])
    mse = np.mean(r*r)
    rmse = np.sqrt(mse)
    mae = np.mean(r)
    return mse, rmse, mae

In [5]:
#Initialize Model
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std = 0.01)
def init():
    net = nn.Sequential(nn.Flatten(), 
                        nn.Linear(num_inputs,num_hidden), 
                        nn.ReLU(), 
                        nn.Dropout(dropout),
                        nn.Linear(num_hidden,num_outputs),
                        nn.Unflatten(1,(num_features, (1+qlen))))
    net.apply(init_weights)
    return net
net = init()
optimizer = torch.optim.SGD(net.parameters(), lr = lr)
def train(model, train_iter, quantiles, loss_fn, optimizer, num_epochs = 100):
    running_loss = 0
    last_loss = 0
    for i, data in enumerate(train_iter):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(quantiles, labels, outputs)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        last_loss = loss.item()
    return last_loss

In [6]:
#Train Model
def iter():
    best_vloss = 1e9
#     animator = d2l.Animator(xlabel = "epoch", ylabel = "loss", xlim = [1,n_epochs], legend = ["train", "val"])
    for epoch in range(n_epochs):
        net.train(True)
        avg_loss = train(net, train_iter, quantiles, lossfn, optimizer, num_epochs = n_epochs)
        net.train(False)
        running_vloss = 0.0
        for i, vdata in enumerate(val_iter):
            vinputs, vlabels = vdata
            voutputs =  net(vinputs)
            vloss = lossfn(quantiles, vlabels, voutputs)
            running_vloss += vloss
        avg_vloss = float(running_vloss / (i+1))
#         if epoch % (n_epochs/20) == 0:
#             animator.add(epoch +1, (avg_loss, avg_vloss))
        if avg_vloss < best_vloss:
            best_vloss = avg_vloss
            torch.save(net, "model_best_state")
    model = torch.load("model_best_state")
    pred = model(X_val).detach().numpy()
    return pred

In [7]:
#Iterate 
cl,tl,x,icp,mil = [],[],[],[[] for _ in range(qlen//2)],[[] for _ in range(qlen//2)]
for _ in range(num_iter):
    net = init()
    optimizer = torch.optim.SGD(net.parameters(), lr = lr)
    pred = iter()
    cl.append(crossing_loss(pred))
    tl.append(tilted_loss(quantiles,Y_val,pred))
    x.append(eval_error(Y_val,pred))
    for i in range(qlen//2):
        t1,t2 = eval_quantiles(pred[:,:,i+1],pred[:,:,qlen-i],Y_val)
        icp[i].append(t1)
        mil[i].append(t2)

In [8]:
#Evaluate Errors
print("Crossing Loss:", np.mean([y[0] for y in cl]), ", Number of Cross:", np.mean([y[1] for y in cl]))
print("MSE:",np.mean([y[0] for y in x]), "RMSE:", np.mean([y[1] for y in x]),"MAE:", np.mean([y[2] for y in x]))
print("Tilted loss:", np.mean(tl))
print("Prediction Intervals:")
for i in range(qlen//2):
    print(round(quantiles[qlen-i-1]-quantiles[i],1)*100,"% ICP & MIL:",round(np.mean(icp[i]),6),round(np.mean(mil[i]),6))
#Test data will be touched after everything is done i.e. tuning hyperparameters & adjusting model architecture

Crossing Loss: 3.4018411928021436e-05 , Number of Cross: 177.83333333333334
MSE: 0.000108547065 RMSE: 0.010388785 MAE: 0.008260885
Tilted loss: 0.18948630287249882
Prediction Intervals:
90.0 % ICP & MIL: 0.96278 0.030739
80.0 % ICP & MIL: 0.882938 0.016032
60.0 % ICP & MIL: 0.70453 0.009112
40.0 % ICP & MIL: 0.499426 0.005204
20.0 % ICP & MIL: 0.28058 0.002318
